In [ ]:
# default_exp data

# Utils

> This module offers useful utilities.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# #hide
# %load_ext autoreload
# %autoreload 2

In [ ]:
#export
import os
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#export
def log_datadir_as_artifact(wandb_run, path_to_dir, artifact_name, artifact_type='dataset'):
    """
    Logs a data directory as an artifact in wandb.
    wandb_run: wandb.Run object
    path_to_dir: path to the data directory
    artifact_name: name of the artifact
    artifact_type: type of the artifact
    """
    
    artifact = wandb.Artifact(artifact_name, type=artifact_type)
    artifact.add_dir(path_to_dir)
    wandb_run.log_artifact(artifact)
    

In [ ]:
#hide
# run = wandb.init(entity='wandb_fc', project='rsna-miccai-brain', group='data', job_type='sample_data')
# log_datadir_as_artifact(run, '../data/sample/', 'sample_data', artifact_type='dataset')
# wandb.finish()

wandb: Adding directory to artifact (.\..\data\sample)... Done. 94.3s


In [ ]:
# #hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_data.ipynb.
Converted 00_utils.ipynb.
Converted index.ipynb.
